In [1]:
import os
print("Init dir: ", os.getcwd())
os.chdir(os.pardir)
print("Work dir: ", os.getcwd())

Init dir:  c:\Users\user20\Desktop\추천시스템\rec_system\py_src
Work dir:  c:\Users\user20\Desktop\추천시스템\rec_system


In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel('raw_data/refined_data.xlsx')
print(df)
df_summary = df.describe()

                                   ID   EXT   NEU   AGR   CON   OPN
0    00419a4c96b32cd63b2c7196da761274  1.90  4.15  3.15  4.65  3.45
1    02c37028a782cfda660c7243e45244bb  2.05  4.00  2.20  2.20  3.75
2    03133a828cd0cf52e3752813ce5d818f  3.84  3.55  2.85  4.32  3.45
3    03e6c4eca4269c183fa0e1780f73faba  3.20  3.60  3.85  4.35  4.80
4    06b055f8e2bca96496514891057913c3  2.85  2.35  3.35  4.70  3.35
..                                ...   ...   ...   ...   ...   ...
245  f83aa7290cf3ae8ed75d6aee5ebbb559  2.65  1.70  3.95  3.00  3.15
246  fbe5aa478508d1dc931427ade5d9e1b4  3.25  2.75  2.25  4.00  4.00
247  fc1c9fb6c64740edcbbf8cfe9dde8b02  3.55  2.25  3.90  2.80  4.00
248  fe22087986fdcc65939c793fe0ec90a9  3.45  2.55  3.15  3.50  2.90
249  fffafe151f07a30a0ede2038a897b680  3.05  1.80  3.35  2.95  4.35

[250 rows x 6 columns]


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# calculation cosine similarity about all items
def cos_sim(data):
    cosine_similarity_matrix = cosine_similarity(data.iloc[:, 1:].values)

    # declare a list to save data
    similarity_results = []

    # for loop for cosine similarity
    for i in range((len(cosine_similarity_matrix))):
        for j in range(i + 1, len(cosine_similarity_matrix)):
            similarity_results.append([i +1, j +1, cosine_similarity_matrix[i, j]])


    # save data as pandas DataFrame
    return pd.DataFrame(similarity_results, columns = ["Row1", "Row2", "Cosine Similarity"])

res_cos = cos_sim(df)
res_cos_summary = res_cos.describe()
print("cosine similarity", res_cos)
print("Summary: ", res_cos_summary)

cosine similarity        Row1  Row2  Cosine Similarity
0         1     2           0.951905
1         1     3           0.966918
2         1     4           0.975332
3         1     5           0.966944
4         1     6           0.958614
...     ...   ...                ...
31120   247   249           0.979177
31121   247   250           0.968925
31122   248   249           0.980273
31123   248   250           0.992880
31124   249   250           0.968722

[31125 rows x 3 columns]
Summary:                 Row1          Row2  Cosine Similarity
count  31125.000000  31125.000000       31125.000000
mean      83.666667    167.333333           0.964076
std       58.807596     58.807596           0.027477
min        1.000000      2.000000           0.785344
25%       34.000000    126.000000           0.951067
50%       74.000000    177.000000           0.971214
75%      125.000000    217.000000           0.984393
max      249.000000    250.000000           0.999927


In [5]:
import random
random.seed(555)
## random shuffle by cosine similarity
print(res_cos_summary)

## summary
mean = res_cos_summary.iloc[1,2]
std = res_cos_summary.iloc[2,2]
print("mean", res_cos_summary.iloc[1,2])
print("std", res_cos_summary.iloc[2,2])

up_lim = mean + std
lo_lim = mean - std
print("up_lim", up_lim)
print("lo_lim", lo_lim)

alpha = 0.05

def cos_shuffle(data, statistical_div):
    '''
    statistical_div represents the level of upper and lower limit
    '''

    filtered_sim = []

    cosine_similarity_matrix = cosine_similarity(data.iloc[:, 1:].values)
    for i in range(len(cosine_similarity_matrix)):
        for j in range(i + 1, len(cosine_similarity_matrix)):
            similarity = cosine_similarity_matrix[i][j]

            ### condition

            # up_lim < sim < max
            if statistical_div == "vh":
                if up_lim < similarity:
                    filtered_sim.append((similarity, i, j))
            
            # mean < sim < up_lim
            if statistical_div == "up":
                if up_lim < similarity:
                    filtered_sim.append((similarity, i, j))

            # lo_lim < sim < up_lim
            if statistical_div == "mid":
                if lo_lim < similarity < up_lim:
                    filtered_sim.append((similarity, i, j))

            # lo_lim < sim < mean
            if statistical_div == "lo":
                if lo_lim < similarity < mean:
                    filtered_sim.append((similarity, i, j))

            # min < sim < lo_lim
            if statistical_div == "vl":
                if similarity < lo_lim:
                    filtered_sim.append((similarity, i, j))

    random_groups = random.sample(filtered_sim, min(6, len(filtered_sim)))
    return random_groups

test = cos_shuffle(df, 'vh')

for idx, (similarity, i, j) in enumerate(test):
    print(f"index {idx}: ID: {i} ID: {j}, similarity: {similarity:.4f}")

               Row1          Row2  Cosine Similarity
count  31125.000000  31125.000000       31125.000000
mean      83.666667    167.333333           0.964076
std       58.807596     58.807596           0.027477
min        1.000000      2.000000           0.785344
25%       34.000000    126.000000           0.951067
50%       74.000000    177.000000           0.971214
75%      125.000000    217.000000           0.984393
max      249.000000    250.000000           0.999927
mean 0.9640764864994569
std 0.027476917070290884
up_lim 0.9915534035697477
lo_lim 0.936599569429166
index 0: ID: 38 ID: 191, similarity: 0.9932
index 1: ID: 55 ID: 99, similarity: 0.9968
index 2: ID: 33 ID: 142, similarity: 0.9967
index 3: ID: 27 ID: 134, similarity: 0.9925
index 4: ID: 125 ID: 151, similarity: 0.9936
index 5: ID: 186 ID: 193, similarity: 0.9976


In [6]:
### paired cosine similarity

import random
from sklearn.metrics.pairwise import cosine_similarity

random.seed(555)

def cos_shuffle(data, statistical_div):
    '''
    statistical_div represents the level of upper and lower limit
    '''

    # calc cosine similarity
    cosine_similarity_matrix = cosine_similarity(data.iloc[:, 1:].values)

    # filtering similarity range
    filtered_sim = []
    for i in range(len(cosine_similarity_matrix)):
        for j in range(i + 1, len(cosine_similarity_matrix)):
            similarity = cosine_similarity_matrix[i][j]

            # condition
            if statistical_div == "vh" and similarity > up_lim:
                filtered_sim.append((similarity, i, j))
            elif statistical_div == "up" and mean < similarity <= up_lim:
                filtered_sim.append((similarity, i, j))
            elif statistical_div == "mid" and lo_lim < similarity <= up_lim:
                filtered_sim.append((similarity, i, j))
            elif statistical_div == "lo" and lo_lim < similarity <= mean:
                filtered_sim.append((similarity, i, j))
            elif statistical_div == "vl" and similarity <= lo_lim:
                filtered_sim.append((similarity, i, j))

    # random sampling from condition range
    random_groups = random.sample(filtered_sim, min(6, len(filtered_sim)))
    return random_groups

# execution function
test = cos_shuffle(df, 'vh')

for similarity, i, j in test:
    print(f"ID: {i} ID: {j}, similarity: {similarity:.4f}")

ID: 38 ID: 191, similarity: 0.9932
ID: 55 ID: 99, similarity: 0.9968
ID: 33 ID: 142, similarity: 0.9967
ID: 27 ID: 134, similarity: 0.9925
ID: 125 ID: 151, similarity: 0.9936
ID: 186 ID: 193, similarity: 0.9976
